# Evaluate trained model

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import cv2
#import numpy as np
#import tensorflow as tf
import sys
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import xml.etree.ElementTree as ET

# Import utilites
#from utils import label_map_util
#from utils import visualization_utils as vis_util

try:
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
#    import labinet.object_detect
except ImportError:
    # This part is only required to run the notebook
    # directory when the module itself is not installed.
    #
    # If you have the module installed, just use "import labinet..."
    import os
    import inspect
    # the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
    cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
    if cmd_subfolder not in sys.path:
        sys.path.insert(0, cmd_subfolder)
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
 #   import labinet.object_detect

%aimport labinet.io_util
%aimport labinet.object_detect

%matplotlib inline

## change path, model name etc to your needs

In [3]:
MODEL_NAME = 'inference_graph'   # the result from Step 6 Export Inference Graph
IMAGE_PATH = 'images'
IMAGE_NAME = '01-20190414173244-01.jpg'   # cmdline arg
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
CWD_PATH = os.path.join(os.getcwd(),'..')  # should become gitbase
NUM_CLASSES = 1

In [4]:
# model
PATH_TO_MODEL = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
# label map
LABEL_MAP = os.path.join(CWD_PATH, 'data', 'object-detection.pbtxt')
# path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH, IMAGE_PATH, IMAGE_NAME)

## Load (frozen) Model into Memory

In [5]:
detection_graph = load_model(PATH_TO_MODEL)
#sess = tf.Session(graph=detection_graph)
   

### load labels 

In [6]:
categories, category_index = load_label_map(LABEL_MAP, NUM_CLASSES)

## Run Inference : Object Detection 
_load the image and run on it the object detection_

In [7]:
# load image
image =  Image.open(PATH_TO_IMAGE)
#imshow(np.asarray(image))

# inference
output_dict = labinet.object_detect.run_inference_for_single_image(image, detection_graph)

### Visualize result

In [9]:
#%aimport labinet.object_detect 

image_with_boxes = labinet.object_detect.visualize_boxes_after_detection(image, output_dict, category_index)
# All the results have been drawn on image. Now display the image.
plt.figure(figsize=IMAGE_SIZE)
#plt.imshow(image_with_boxes)
print(IMAGE_NAME)


01-20190414173244-01.jpg


<Figure size 864x576 with 0 Axes>

In [ ]:
boxes = output_dict['detection_boxes']
scores = output_dict['detection_scores']


In [ ]:
def get_boxes_to_use(detection_boxes, detection_scores, min_score_threshold=0.8):
    boxes = detection_boxes
    scores = detection_scores
    #print(output_dict['detection_boxes'])
    num_boxes = boxes.shape[0]    
    boxes_to_use = []
    for i in range(num_boxes):
        if scores is None or scores[i] > min_score_threshold:
            box = boxes[i]
            boxes_to_use.append(box)       
            #score = int(scores[i] * 100)
            #print(f"Use box[{i}]={boxes[i]}. (score is {score}%)")
    return boxes_to_use        
    

In [ ]:
boxes_to_use = get_boxes_to_use(boxes, scores)
boxes_to_use[0]

In [ ]:
def get_normalized_coordinates(box, image_size):
    '''
    return box coordinates resized to image_size
    left, right, top, bottom aka xmin, xmax, ymin, ymax
    '''
    (ymin, xmin, ymax, xmax) = box
    im_width = image_size[0]
    im_height = image_size[1]
    (left, right, top, bottom) = (int(xmin * im_width), int(xmax * im_width), int(ymin * im_height), int(ymax * im_height))
    box = { 'xmin': left, 'xmax': right, 'ymin': top, 'ymax': bottom }
    return box
    

In [ ]:
boxes_normed = []
for box in boxes_to_use:
    box_normed = get_normalized_coordinates(box, image.size)
    boxes_normed.append(box_normed)

In [ ]:
def box_to_labelimg_xml(filename, image_size, boxes, label='dog', path=None, ):
    # todo: change xmin, xmax... to array of 'boxes'
    root = ET.Element('annotation')
    folder = ET.SubElement(root, 'folder')
    folder.text = 'images'
    fname = ET.SubElement(root, 'filename')
    fname.text = filename
    
    source = ET.SubElement(root, 'source')
    database = ET.SubElement(source, 'database')
    database.text = 'Unknown'
    
    size = ET.SubElement(root, 'size')
    width = ET.SubElement(size, 'width')
    width.text = str(image_size[0])
    height = ET.SubElement(size, 'height')
    height.text = str(image_size[1])
    depth = ET.SubElement(size, 'depth')
    depth.text = '3'
    
    segmented = ET.SubElement(root, 'segmented')
    segmented.text = '0'
    
    ### 1 box. each box becomes 'object'
    for box in boxes:
        obj = ET.SubElement(root, 'object')
        name = ET.SubElement(obj, 'name')
        name.text = label
        pose = ET.SubElement(obj, 'pose')
        pose.text = 'Unspecified'
        truncated = ET.SubElement(obj, 'truncated')
        truncated.text = '0'
        difficult = ET.SubElement(obj, 'difficult')
        difficult.text = '0'
        bndbox = ET.SubElement(obj, 'bndbox')
        xminx = ET.SubElement(bndbox, 'xmin')
        xminx.text = str(box['xmin'])
        xmaxx = ET.SubElement(bndbox, 'xmax')
        xmaxx.text = str(box['xmax'])
        yminx = ET.SubElement(bndbox, 'ymin')
        yminx.text = str(box['ymin'])
        ymaxx = ET.SubElement(bndbox, 'ymax')
        ymaxx.text = str(box['ymax'])

    tree = ET.ElementTree(root)
    return tree
    

In [ ]:
xml = box_to_labelimg_xml('file.jpg', (640,480), boxes_normed)
ET.dump(xml)
#xml.tostring()